In [2]:
cd ../

C:\Users\teruto\Desktop\livedoor-news-clustering


In [4]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import math

In [5]:
#ファイル読み込み
train_df = pd.read_csv('./data/preprosessing_train.csv')
X_df_train = train_df[['text_length', 'is_year', 'is_month', 'is_date',
       'is_time', 'num_hatena', 'num_bikkuri', 'is_eiga', 'is_henkakko',
       'is_normalkakko', 'is_whitekakko', 'is_vol', 'is_sports',
       'text_del_one']]
Y_df_train = train_df["flg"].astype(np.int64)
X_train =  X_df_train.values
t_train = Y_df_train.values

dev_df = pd.read_csv('./data/preprosessing_dev.csv')
X_df_dev = dev_df[['text_length', 'is_year', 'is_month', 'is_date',
       'is_time', 'num_hatena', 'num_bikkuri', 'is_eiga', 'is_henkakko',
       'is_normalkakko', 'is_whitekakko', 'is_vol', 'is_sports',
       'text_del_one']]
Y_df_dev = dev_df["flg"].astype(np.int64)
X_valid =  X_df_dev.values
t_valid = Y_df_dev.values

test_df = pd.read_csv('./data/preprosessing_test.csv')
X_df_test = test_df[['text_length', 'is_year', 'is_month', 'is_date',
       'is_time', 'num_hatena', 'num_bikkuri', 'is_eiga', 'is_henkakko',
       'is_normalkakko', 'is_whitekakko', 'is_vol', 'is_sports',
       'text_del_one']]
Y_df_test = test_df["flg"].astype(np.int64)
X_test =  X_df_test.values
t_test = Y_df_test.values

In [6]:
#特徴量配列とテキスト配列に分ける
X_train_feat = X_train[:,:-1]
X_valid_feat = X_valid[:,:-1]
X_test_feat = X_test[:,:-1]
X_train_feat.shape
X_train_text = X_train[:,-1]
X_valid_text = X_valid[:,-1]
X_test_text = X_test[:,-1]

In [7]:
#特徴量標準化
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
ss = MinMaxScaler()
X_train_feat_ss = ss.fit_transform(X_train_feat)
X_valid_feat_ss = ss.transform(X_valid_feat)
X_test_feat_ss = ss.transform(X_test_feat)

In [8]:
#文字カラムに対する欠損値削除
X_train_del_one = X_train_text
for idx in range(len(X_train_del_one)):
    if type(X_train_del_one[idx]) is float:
        X_train_del_one[idx] = "Null" 

X_valid_del_one = X_valid_text
for idx in range(len(X_valid_del_one)):
    if type(X_valid_del_one[idx]) is float:
        X_valid_del_one[idx] = "Null"
        
X_test_del_one = X_test_text
for idx in range(len(X_test_del_one)):
    if type(X_test_del_one[idx]) is float:
        X_test_del_one[idx] = "Null"

In [9]:
#bag of words による単語ベクトル化(最大値、最小値を設定した)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
cv = TfidfVectorizer(min_df=3/5000, max_df=3000/5000)
#cv = TfidfVectorizer()
X_train_cv = cv.fit_transform(X_train_del_one).toarray().astype(np.float32)
X_valid_cv = cv.transform(X_valid_del_one).toarray().astype(np.float32)
X_test_cv = cv.transform(X_test_del_one).toarray().astype(np.float32)

In [3]:
#word2vecによる単語ベクトル化(モデルのロード)
import gensim
model= gensim.models.KeyedVectors.load_word2vec_format('./w2v/jawiki.word_vectors.300d.abtt.bin', binary=True)

In [10]:
model_vocab = set(model.vocab.keys())

In [11]:
#word2vecによる単語ベクトル化(モデルのロード)
def swem_avg(word_list,model, model_vocab):
    ave_arr = np.mean(np.array([model[word] for word in word_list.split(" ") if word in model_vocab]), axis=0)
    if np.shape(ave_arr) != (300,):
        return np.zeros((1,300))
    return ave_arr.reshape([1,300])

X_train_w2v = np.zeros((1,300))
for text in X_train_del_one:
    X_train_w2v = np.concatenate([X_train_w2v,swem_avg(text, model, model_vocab)])
X_train_w2v = X_train_w2v[1:len(X_train_w2v)].astype(np.float32)
X_valid_w2v = np.zeros((1,300))
for text in X_valid_del_one:
    X_valid_w2v = np.concatenate([X_valid_w2v,swem_avg(text, model, model_vocab)])
X_valid_w2v = X_valid_w2v[1:len(X_valid_w2v)].astype(np.float32)
X_test_w2v = np.zeros((1,300))
for text in X_test_del_one:
    X_test_w2v = np.concatenate([X_test_w2v,swem_avg(text, model, model_vocab)])
X_test_w2v = X_test_w2v[1:len(X_test_w2v)].astype(np.float32)

c:\users\teruto\desktop\sourcecode\livedoor-news-clustering\venv\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\teruto\desktop\sourcecode\livedoor-news-clustering\venv\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [12]:
print(np.shape(X_train_w2v))
print(np.shape(X_valid_w2v))
print(np.shape(X_test_w2v))

(5000, 300)
(500, 300)
(500, 300)


In [13]:
type(X_train_w2v[0])

numpy.ndarray

In [23]:
#文字配列と特徴量の配列を結合する

X_train = np.concatenate([X_train_cv,X_train_feat_ss,X_train_w2v],1)
X_valid = np.concatenate([X_valid_cv,X_valid_feat_ss,X_valid_w2v],1)
X_test = np.concatenate([X_test_cv,X_test_feat_ss,X_test_w2v],1)
"""
X_train = np.concatenate([X_train_feat_ss,X_train_w2v],1)
X_valid = np.concatenate([X_valid_feat_ss,X_valid_w2v],1)
X_test = np.concatenate([X_test_feat_ss,X_test_w2v],1)
"""

MemoryError: Unable to allocate 128. MiB for an array with shape (5000, 3350) and data type float64

In [24]:
#型変換
X_train = X_train.astype(np.float32)
X_valid = X_valid.astype(np.float32)
X_test = X_test.astype(np.float32)
t_train = t_train.astype(np.int32)
t_valid = t_valid.astype(np.int32)
t_test = t_test.astype(np.int32)

ValueError: could not convert string to float: '本田 ｲﾁﾛｰ 似る 作文 話題 必ず 世界一 なる '

In [25]:

#スパース配列に変換
from scipy.sparse import csr_matrix
X_train_csr = csr_matrix(X_train)
X_valid_csr = csr_matrix(X_valid)
X_test_csr = csr_matrix(X_test)

TypeError: no supported conversion for types: (dtype('O'),)

In [156]:
#混合行列
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score ,precision_score, recall_score,f1_score
import seaborn as sns
def result_heatmap(Y_test,Y_pred):
    print("正解率:"+str(accuracy_score(Y_test, Y_pred)))
    cm = confusion_matrix(Y_test, Y_pred)
    #print(cm)
    sns.heatmap(cm,annot=True, cmap='Blues')

In [157]:
#PCA
from sklearn.decomposition import  TruncatedSVD 
pca =  TruncatedSVD(500)

X_train_pca = pca.fit_transform(X_train_csr) 
X_valid_pca = pca.transform(X_valid_csr)
X_test_pca = pca.transform(X_test_csr)
"""
X_train_pca = X_train_csr.toarray()
X_valid_pca = X_valid_csr.toarray()
X_test_pca = X_test_csr.toarray()
"""

'\nX_train_pca = X_train_csr.toarray()\nX_valid_pca = X_valid_csr.toarray()\nX_test_pca = X_test_csr.toarray()\n'

In [158]:
#models
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [159]:


class S1Model():
    def __init__(self,name):
        self.models = []
        self.name = name
    

        
    def fit_proba(self,X_train,y_train,X_valid):
        model = self.get_classifier(self.name)
        model.fit(X_train,y_train)
        self.models.append(model)
        return model.predict_proba(X_valid)
    
    def predict(self,X_test):  
        ave_arr = np.zeros((len(X_test),9))
        for model in self.models:
            proba = model.predict_proba(X_test)
            np.add(ave_arr,proba)
            proba = model.predict_proba(X_test_pca)
            ave_arr = np.add(ave_arr,proba)
        return ave_arr/len(self.models)
    
    def get_classifier(self,name):
        if name == "svm1":
            return svm.SVC(kernel="rbf",gamma = 0.01,C=50,probability=True)
        elif name == "svm2":
            return svm.SVC(kernel="linear",gamma = 10,C=10,probability=True)
        elif name == "rfc1":
            return RandomForestClassifier(n_estimators=100,max_depth = 3,criterion="entropy")
        elif name == "rfc2":
            return RandomForestClassifier(n_estimators=500,max_depth = 5,criterion= "gini")
        elif name == "mlp":
            return MLPClassifier(hidden_layer_sizes=(256,128,64,32),activation="relu",solver = "adam",max_iter=10000,early_stopping=True)
        elif name == "lr":
            return LogisticRegression(C=300,random_state=1)
        elif name == "gbc":
            return GradientBoostingClassifier(random_state=0)
        elif name == "abc":
            return AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=0), n_estimators=100, learning_rate=1,  random_state=None)
        elif name == "k-nei1":
            return  KNeighborsClassifier(n_neighbors=5)
        elif name == "k-nei2":
            return  KNeighborsClassifier(n_neighbors=20)
        

In [ ]:
from sklearn.model_selection import StratifiedKFold
from copy import deepcopy

S1_models = [S1Model("svm1"),S1Model("svm2"),S1Model("rfc1"),S1Model("rfc2"),S1Model("mlp"),S1Model("lr"),S1Model("k-nei1"),S1Model("k-nei2"),S1Model("gbc"),S1Model("abc")]

X_blends = np.zeros((1,9*len(S1_models)))#9はラベルの種類
y_blends = np.zeros(1)

skf = StratifiedKFold(n_splits=4)#ここのランダム大事よ
for train_idx, valid_idx in skf.split(X_train_pca,t_train):
    print("-------------------")
    X_train = X_train_pca[train_idx]
    y_train = t_train[train_idx]
    X_valid = X_train_pca[valid_idx]
    y_valid = t_train[valid_idx]
    temp_X_blends = np.zeros((np.shape(X_valid)[0],1))
    for model in S1_models:
        proba = model.fit_proba(X_train,y_train,X_valid)
        temp_X_blends = np.concatenate([temp_X_blends,proba],1)
        print(model.name+"done")
    #結合
    X_blends = np.concatenate([X_blends,temp_X_blends[:,1:len(temp_X_blends)]],0)
    y_blends = np.append(y_blends, y_valid)
X_blends = X_blends[1:np.shape(X_blends)[0],:]
y_blends = y_blends[1:len(y_blends)]

-------------------
svm1done
svm2done
rfc1done
rfc2done
mlpdone


c:\users\teruto\desktop\sourcecode\livedoor-news-clustering\venv\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


lrdone
k-nei1done
k-nei2done
gbcdone
abcdone
-------------------
svm1done
svm2done
rfc1done
rfc2done
mlpdone


c:\users\teruto\desktop\sourcecode\livedoor-news-clustering\venv\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


lrdone
k-nei1done
k-nei2done
gbcdone
abcdone
-------------------
svm1done
svm2done
rfc1done
rfc2done
mlpdone


c:\users\teruto\desktop\sourcecode\livedoor-news-clustering\venv\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


lrdone
k-nei1done
k-nei2done
gbcdone
abcdone
-------------------
svm1done
svm2done
rfc1done
rfc2done
mlpdone


c:\users\teruto\desktop\sourcecode\livedoor-news-clustering\venv\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


lrdone
k-nei1done
k-nei2done


In [ ]:
np.shape(y_valids)

In [ ]:
#2層目
#best: S2_model =svm.SVC(kernel="rbf",degree = 10,gamma = 0.01,C=10,probability=True)
S2_model =LogisticRegression(C=300,random_state=1)
X_S1_train = np.concatenate([X_blends,X_train_pca],1)
print(np.shape(X_valids))
S2_model.fit(X_S1_train,y_blends)


In [ ]:
len(X_test_pca)

In [ ]:
#Stage1の推論
X_S2_test = np.zeros((np.shape(X_test_pca )[0],1)) 
for S1_model in S1_models:
    proba = S1_model.predict(X_test_pca)
    print(S1_model.name)
    X_S2_test = np.concatenate([X_S2_test,proba],1)
X_S2_test= X_S2_test[:,1:len(X_S2_test)]

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score ,precision_score, recall_score,f1_score
import seaborn as sns
def result_heatmap(Y_test,Y_pred):
    print("正解率:"+str(accuracy_score(Y_test, Y_pred)))
    cm = confusion_matrix(Y_test, Y_pred)
    #print(cm)
    sns.heatmap(cm,annot=True, cmap='Blues')

In [ ]:

X_S2_test = np.concatenate([X_S2_test,X_test_pca],1)
t_S2_pred = S2_model.predict(X_S2_test)
result_heatmap(t_test,t_S2_pred)